In [ ]:
! pip install ipywidgets

In [13]:
import pandas as pd

In [14]:
df = pd.read_csv("backup_mod_37.csv")

In [15]:
df.head()

,Unnamed: 0,Serial No.,Reaction Type,Description,General Formula,Source,Generated Prompt
0,0,1,Synthesis/Combination Reaction,Two or more reactants combine to form a single...,A + B → AB,https://en.wikipedia.org/wiki/Chemical_reaction,A synthesis (or combination) reaction involves...
1,1,2,Decomposition Reaction,A single compound breaks down into two or more...,AB → A + B,https://en.wikipedia.org/wiki/Chemical_reaction,A decomposition reaction involves a complex su...
2,2,3,Single Displacement/Substitution,A more reactive element displaces a less react...,A + BC → AC + B,https://en.wikipedia.org/wiki/Chemical_reaction,"In a single displacement reaction, an uncombin..."
3,3,4,Double Displacement/Substitution,Two compounds exchange ions to produce two new...,AB + CD → AD + CB,https://en.wikipedia.org/wiki/Chemical_reaction,Double Displacement Reaction Summary:\n\nIn a ...
4,4,5,Combustion Reaction,"A substance reacts with oxygen, releasing ener...",CₓHᵧ + O₂ → CO₂ + H₂O + energy,https://en.wikipedia.org/wiki/Chemical_reaction,"In a combustion reaction, an element or compou..."


In [16]:
def discover_modify_generated_prompt(df):
    """
    This function takes a dataframe and shows the reaction_type and the generated_prompt iteratively.
    And allows user to modify the generated_prompt if needed. (if returned value is empty or " ", it is not modified)
    and at the end it returns the modified dataframe.
    Instead of normal user input, try ipython version of input to allow better editing experience in jupyter notebook.
    """
    from IPython.display import display, clear_output
    from ipywidgets import Textarea, Button, VBox, HBox, Output

    modified_prompts = []
    out = Output()
    result_df = df.copy()  # Create a copy to modify

    def on_button_click(b):
        with out:
            out.clear_output()
            idx = int(b.description)
            print(f"Processing item {idx + 1} of {len(df)}")
            print(f"Reaction Type: {df.loc[idx, 'Reaction Type']}")
            print(f"Original Prompt: {df.loc[idx, 'Generated Prompt']}")
            textarea = Textarea(value=df.loc[idx, 'Generated Prompt'], layout={'width': '1000px', 'height': '200px'})
            save_button = Button(description="Save", button_style='success')
            cancel_button = Button(description="Cancel", button_style='warning')

            def on_save_click(sb):
                modified_value = textarea.value.strip()
                if modified_value:
                    modified_prompts.append((idx, modified_value))
                else:
                    modified_prompts.append((idx, df.loc[idx, 'Generated Prompt']))
                out.clear_output()
                display_next()

            def on_cancel_click(cb):
                modified_prompts.append((idx, df.loc[idx, 'Generated Prompt']))
                out.clear_output()
                display_next()

            save_button.on_click(on_save_click)
            cancel_button.on_click(on_cancel_click)

            display(VBox([textarea, HBox([save_button, cancel_button])]))

    def display_next():
        if len(modified_prompts) < len(df):
            next_idx = len(modified_prompts)
            button = Button(description=str(next_idx), button_style='info')
            button.on_click(on_button_click)
            with out:
                out.clear_output()
                print(f"Click to process item {next_idx + 1} of {len(df)}")
                display(button)
        else:
            # All prompts processed
            with out:
                out.clear_output()
                print("All prompts processed!")
                
                # Apply modifications to the result dataframe
                for idx, new_prompt in modified_prompts:
                    result_df.at[idx, 'Generated Prompt'] = new_prompt
                
                print("Modified DataFrame:")
                display(result_df.head())
                
                # Store the result in a global variable for access
                global processed_df
                processed_df = result_df
                print("\nDataFrame has been saved to 'processed_df' variable.")

    display_next()
    display(out)
    return result_df

In [17]:
res_df = discover_modify_generated_prompt(df)

Output()

In [18]:
processed_df.to_csv("done_df.csv", index=False)

In [21]:
new_df = processed_df.iloc[:, 1:]

In [22]:
new_df.head()

,Serial No.,Reaction Type,Description,General Formula,Source,Generated Prompt
0,1,Synthesis/Combination Reaction,Two or more reactants combine to form a single...,A + B → AB,https://en.wikipedia.org/wiki/Chemical_reaction,A synthesis (or combination) reaction involves...
1,2,Decomposition Reaction,A single compound breaks down into two or more...,AB → A + B,https://en.wikipedia.org/wiki/Chemical_reaction,A decomposition reaction involves a complex su...
2,3,Single Displacement/Substitution,A more reactive element displaces a less react...,A + BC → AC + B,https://en.wikipedia.org/wiki/Chemical_reaction,"In a single displacement reaction, an uncombin..."
3,4,Double Displacement/Substitution,Two compounds exchange ions to produce two new...,AB + CD → AD + CB,https://en.wikipedia.org/wiki/Chemical_reaction,Double Displacement Reaction Summary:\n\nIn a ...
4,5,Combustion Reaction,"A substance reacts with oxygen, releasing ener...",CₓHᵧ + O₂ → CO₂ + H₂O + energy,https://en.wikipedia.org/wiki/Chemical_reaction,"In a combustion reaction, an element or compou..."


In [23]:
new_df.to_csv("final_all_eq_added.csv", index=False)